# Stable Coastal Wetlands 
Created by Blair Tirpak in 2022. Updated 5/19/2023 to add values for unstable coastal wetlands and stables wetlands with high uncertainty as well as overall extent. Updated 8/29/2023 to update color and legend. 

Calcuate standard deviation of UVVR yearly rasters to calc confidence

## GOM

In [8]:
# import arcpy and set enviorments

import arcpy
from arcpy import env
from arcpy.sa import *

# set enviorments
env.workspace = r"C:\Data\Indicators\CoastalMarsh\CoastalMarsh2023.gdb"

# Final Extent
extent = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# define final outputs
SEout = r"C:\Data\Indicators\CoastalMarsh\CoastalMarsh\StableCoastalWetlands.tif"

In [8]:
# extract UVVR band from composite raster
arcpy.env.snapRaster = r"C:\Data\Indicators\CoastalMarsh\GOM\L8_2014_GOM_Unveg_Veg_Water_UVVR.tif"
arcpy.env.cellSize =  r"C:\Data\Indicators\CoastalMarsh\GOM\L8_2014_GOM_Unveg_Veg_Water_UVVR.tif"

G14 = arcpy.ia.ExtractBand( r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SaltMarsh\L8_2014_GOM_Unveg_Veg_Water_UVVR.tif", 4)
G14.save("G14")
G15 = arcpy.ia.ExtractBand( r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SaltMarsh\L8_2015_GOM_Unveg_Veg_Water_UVVR.tif", 4)
G15.save("G15")
G16 = arcpy.ia.ExtractBand( r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SaltMarsh\L8_2016_GOM_Unveg_Veg_Water_UVVR.tif", 4)
G16.save("G16")
G17 = arcpy.ia.ExtractBand( r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SaltMarsh\L8_2017_GOM_Unveg_Veg_Water_UVVR.tif", 4)
G17.save("G17")
G18 = arcpy.ia.ExtractBand( r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SaltMarsh\L8_2018_GOM_Unveg_Veg_Water_UVVR.tif", 4)
G18.save("G18")

In [9]:
# calculate mean and SD of UVVR 2014-2018

Gmean = arcpy.sa.CellStatistics("G14;G15;G16;G17;G18", "MEAN", "NODATA", "SINGLE_BAND"); 
Gmeanr.save("Gmean")                    

Gsd = arcpy.sa.CellStatistics("G14;G15;G16;G17;G18", "STD", "NODATA", "SINGLE_BAND"); 
Gsd.save("Gsd")

In [2]:
## extract stable marsh mean and SD <= 0.15

# mulitple raster by 1000 to convert to integer
Gmean_1000 = arcpy.sa.Times("Gmean", 1000); Gmean_1000.save("Gmean_1000")
    
Gsd_1000 = arcpy.sa.Times("Gsd", 1000); Gsd_1000.save("Gsd_1000")

# convert to integer
Gmean_1000_int = arcpy.ia.Int("Gmean_1000"); Gmean_1000_int.save("Gmean_1000_int")

Gsd_1000_int = arcpy.ia.Int("Gsd_1000"); Gsd_1000_int.save("Gsd_1000_int")

# extract only stable marsh; <= 0.15
Gmean_1000_int_15 = arcpy.sa.Reclassify("Gmean_1000_int", "Value", "0 150 1;151 3203996160 NODATA", "DATA"); Gmean_1000_int_15.save("Gmean_1000_int_15")

Gsd_1000_int_15 = arcpy.sa.Reclassify("Gsd_1000_int", "Value", "0 150 1;151 4090510592 NODATA", "DATA"); Gsd_1000_int_15.save("Gsd_1000_int_15")

In [3]:
# find areas that meet both thresholds; stable marsh - full extent
GOM_StableMarsh_all = arcpy.sa.Times("Gmean_1000_int_15", "Gsd_1000_int_15"); GOM_StableMarsh_all.save("GOM_StableMarsh_all")

In [5]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

In [4]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# project rasters

arcpy.management.ProjectRaster("GOM_StableMarsh_all", "GOM_StableMarsh_allP", sr,  "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")

<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\GOM_StableMarsh_allP'>

### create GOM Extent RAster

In [ ]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# project rasters
#arcpy.management.ProjectRaster("Gmean", "GmeanP", sr, "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")
arcpy.management.ProjectRaster("Gsd", "GsdP", sr,  "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")

In [5]:
## For reference
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

### Make GOM Extent raster

In [7]:
# Reclass to 0 
GMax = arcpy.GetRasterProperties_management("GsdP", "MAXIMUM")
GMax

remap = RemapRange([[0, GMax, 0]]) 

GOM_extent = arcpy.sa.Reclassify("GsdP", "VALUE", remap, "DATA"); 
GOM_extent.save("GOM_extent")

## ATL 

In [20]:
# import arcpy and set enviorments

import arcpy
from arcpy import env
from arcpy.sa import *

# set enviorments
env.workspace = r"F:\Blueprint\2023Indicators\CoastalMarsh\CoastalMarsh2023.gdb"

arcpy.env.snapRaster = r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2014_ATL_Unveg_Veg_Water_UVVR.tif"
arcpy.env.cellSize = r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2014_ATL_Unveg_Veg_Water_UVVR.tif"

In [19]:
# extract UVVR band from composite raster
A14 = arcpy.ia.ExtractBand(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2014_ATL_Unveg_Veg_Water_UVVR.tif", 4)
A14.save("A14")

A15 = arcpy.ia.ExtractBand(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2015_ATL_Unveg_Veg_Water_UVVR.tif", 4)
A15.save("A15")

A16 = arcpy.ia.ExtractBand(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2016_ATL_Unveg_Veg_Water_UVVR.tif", 4)
A16.save("A16")

A17 = arcpy.ia.ExtractBand(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2017_ATL_Unveg_Veg_Water_UVVR.tif", 4)
A17.save("A17")

A18 = arcpy.ia.ExtractBand(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\ATL\L8_2018_ATL_Unveg_Veg_Water_UVVR.tif", 4)
A18.save("A18")

In [20]:
# calculate mean and SD of UVVR 2014-2018
AMean = arcpy.sa.CellStatistics("A14;A15;A16;A17;A18", "MEAN", "NODATA", "SINGLE_BAND"); 
AMean.save("AMean")                    

Asd = arcpy.sa.CellStatistics("A14;A15;A16;A17;A18", "STD", "NODATA", "SINGLE_BAND"); 
Asd.save("Asd")

In [11]:
# reclassify rasters
Amean_1000 = arcpy.sa.Times("Amean", 1000); Amean_1000.save("Amean_1000")    
Asd_1000 = arcpy.sa.Times("Asd", 1000); Asd_1000.save("Asd_1000")

In [13]:
# convert to integer
Amean_1000_int = arcpy.ia.Int("Amean_1000"); Amean_1000_int.save("Amean_1000_int")

Asd_1000_int = arcpy.ia.Int("Asd_1000"); Asd_1000_int.save("Asd_1000_int")

In [21]:
 # extract only stable marsh; <= 0.15
Amean_1000_int_15 = arcpy.sa.Reclassify("Amean_1000_int", "Value", "0 150 1;151 4210423808 NODATA", "DATA"); 
Amean_1000_int_15.save("Amean_1000_int_15")

Asd_1000_int_15 = arcpy.sa.Reclassify("Asd_1000_int", "Value", "0 150 1;151 4124790784 NODATA", "DATA"); 
Asd_1000_int_15.save("Asd_1000_int_15")

In [22]:
# find areas that meet both thresholds

ATL_StableMarsh_all = arcpy.sa.Times("Amean_1000_int_15", "Asd_1000_int_15"); ATL_StableMarsh_all.save("ATL_StableMarsh_all")

In [23]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# project rasters
arcpy.management.ProjectRaster("ATL_StableMarsh_all", "ATL_StableMarsh_allP", sr,  "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\ATL_StableMarsh_allP'>

In [24]:
# clip ATL rasters to BP extent
ATL_StableMarsh_all_SE = ExtractByMask("ATL_StableMarsh_allP", extent); ATL_StableMarsh_all_SE.save("ATL_StableMarsh_all_SE")

In [14]:
#### NEED THIS???
#out_raster = arcpy.sa.Reclassify("ATL_StableMarsh_all", "Value", "0 10 1;100 100 2", "DATA"); 
#out_raster.save("ATL_StableMarsh_full")


### Make ATL Extent raster

In [26]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# project rasters
#arcpy.management.ProjectRaster("Amean", "AmeanP", sr, "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")
arcpy.management.ProjectRaster("Asd", "AsdP", sr,  "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")

# clip to SE BP extent
AsdP_SE = ExtractByMask("AsdP", extent); AsdP_SE.save("AsdP_SE")

In [27]:
# clip to SE BP extent
AsdP_SE = ExtractByMask("AsdP", extent); AsdP_SE.save("AsdP_SE")

In [28]:
# Reclass SD raster to 0 for extent
# identify max for reclass
AMax = arcpy.GetRasterProperties_management("AsdP_SE", "MAXIMUM")
AMax

# set up local variables
remap = RemapRange([[0, AMax, 0]]) 

# Execute Reclassify
ATL_extent = Reclassify("AsdP_SE", "VALUE", remap)
ATL_extent.save("ATL_extent")


In [32]:
## Merge Extents
arcpy.MosaicToNewRaster_management("GOM_extent;ATL_extent", env.workspace, "SE_extent",  sr,\
                                   "8_BIT_UNSIGNED", "30", "1", "MAXIMUM","FIRST")

<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_extent'>

## NWI formating

### Extract wetlands

In [26]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\TX_geodatabase_wetlands.gdb\TX_Wetlands", "TX_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\TX_Wetlands_coastal'>

In [28]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\LA_geodatabase_wetlands.gdb\LA_Wetlands", "LA_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\LA_Wetlands_coastal'>

In [40]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\MS_geodatabase_wetlands.gdb\MS_Wetlands", "MS_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\MS_Wetlands_coastal'>

In [29]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\AL_geodatabase_wetlands.gdb\AL_Wetlands", "AL_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\AL_Wetlands_coastal'>

In [32]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\FL_geodatabase_wetlands.gdb\FL_Wetlands", "FL_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\FL_Wetlands_coastal'>

In [33]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\GA_geodatabase_wetlands.gdb\GA_Wetlands", "GA_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\GA_Wetlands_coastal'>

In [34]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\SC_geodatabase_wetlands.gdb\SC_Wetlands", "SC_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SC_Wetlands_coastal'>

In [35]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\NC_geodatabase_wetlands.gdb\NC_Wetlands", "NC_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\NC_Wetlands_coastal'>

In [36]:
# Select for coastal wetlands including estuarine intertidal and tidal and freshwater wetlands

arcpy.analysis.Select(r"F:\Blueprint\2022\2022Indicators\SaltMarshIndex\VA_geodatabase_wetlands.gdb\VA_Wetlands", "VA_Wetlands_coastal", """ATTRIBUTE LIKE 'E2%' 
OR 
(
ATTRIBUTE LIKE '%L' OR ATTRIBUTE LIKE '%N' OR ATTRIBUTE LIKE '%P' OR ATTRIBUTE LIKE '%M' OR ATTRIBUTE LIKE '%S' OR ATTRIBUTE LIKE '%R' OR ATTRIBUTE LIKE '%T' OR ATTRIBUTE LIKE '%V' 
OR 
( 
(
ATTRIBUTE LIKE '%L_' OR ATTRIBUTE LIKE '%N_' OR ATTRIBUTE LIKE '%P_' OR ATTRIBUTE LIKE '%M_' OR ATTRIBUTE LIKE '%S_' OR ATTRIBUTE LIKE '%R_' OR ATTRIBUTE LIKE '%T_' OR ATTRIBUTE LIKE '%V_' 
OR 
ATTRIBUTE LIKE '%L__' OR ATTRIBUTE LIKE '%N__' OR ATTRIBUTE LIKE '%P__' OR ATTRIBUTE LIKE '%M__' OR ATTRIBUTE LIKE '%S__' OR ATTRIBUTE LIKE '%R__' OR ATTRIBUTE LIKE '%T__' OR ATTRIBUTE LIKE '%V__' 
)
AND 
(
ATTRIBUTE LIKE '%b' OR ATTRIBUTE LIKE '%d' OR ATTRIBUTE LIKE '%f' OR ATTRIBUTE LIKE '%h' OR ATTRIBUTE LIKE '%r' OR ATTRIBUTE LIKE '%s' OR ATTRIBUTE LIKE '%x' 
OR 
ATTRIBUTE LIKE '%1' OR ATTRIBUTE LIKE '%2' OR ATTRIBUTE LIKE '%3' OR ATTRIBUTE LIKE '%4' OR ATTRIBUTE LIKE '%5' OR ATTRIBUTE LIKE '%6' OR ATTRIBUTE LIKE '%7' OR ATTRIBUTE LIKE '%8' OR ATTRIBUTE LIKE '%9' 
) 
)
AND NOT 
(
ATTRIBUTE LIKE 'PUSb%' OR ATTRIBUTE LIKE 'PUSd%' OR ATTRIBUTE LIKE 'PUSf%' OR ATTRIBUTE LIKE 'PUSh%' OR ATTRIBUTE LIKE 'PUSr%' OR ATTRIBUTE LIKE 'PUSs%' OR ATTRIBUTE LIKE 'PUSx%' 
OR 
ATTRIBUTE LIKE 'L%USb%' OR ATTRIBUTE LIKE 'L%USd%' OR ATTRIBUTE LIKE 'L%USf%' OR ATTRIBUTE LIKE 'L%USh%' OR ATTRIBUTE LIKE 'L%USr%' OR ATTRIBUTE LIKE 'L%USs%' OR ATTRIBUTE LIKE 'L%USx%' 
) 
OR 
(
ATTRIBUTE LIKE 'R1%' AND (ATTRIBUTE LIKE '%EM%' OR ATTRIBUTE LIKE '%US%')
) 
) 
AND NOT 
(
ATTRIBUTE LIKE '%A_' OR ATTRIBUTE LIKE '%C_' OR ATTRIBUTE LIKE '%E_' OR ATTRIBUTE LIKE '%F_' OR ATTRIBUTE LIKE '%G_' OR ATTRIBUTE LIKE '%H_' OR ATTRIBUTE LIKE '%J_' OR ATTRIBUTE LIKE '%K_' 
OR 
ATTRIBUTE LIKE '%C__' OR ATTRIBUTE LIKE '%E__' OR ATTRIBUTE LIKE '%F__' OR ATTRIBUTE LIKE '%G__' OR ATTRIBUTE LIKE '%H__' OR ATTRIBUTE LIKE '%J__' OR ATTRIBUTE LIKE '%K__' 
) 
AND NOT
(
ATTRIBUTE LIKE '_b' OR ATTRIBUTE LIKE '_d' OR ATTRIBUTE LIKE '_f' OR ATTRIBUTE LIKE '_h' OR ATTRIBUTE LIKE '_r' OR ATTRIBUTE LIKE '_s' OR ATTRIBUTE LIKE '_x' 
)
AND ATTRIBUTE NOT LIKE '__S' 
AND ATTRIBUTE NOT LIKE '__M' 
AND ATTRIBUTE NOT LIKE '_'""")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\VA_Wetlands_coastal'>

### Merger and Convert to raster

In [41]:
# merge coastal wetlands
arcpy.Merge_management(["TX_Wetlands_coastal", "LA_Wetlands_coastal", "MS_Wetlands_coastal", "AL_Wetlands_coastal", "FL_Wetlands_coastal", "GA_Wetlands_coastal", "SC_Wetlands_coastal", "NC_Wetlands_coastal", "VA_Wetlands_coastal"], 
                       "SE_Wetlands_coastal")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_Wetlands_coastal'>

In [42]:
# dissolve coastal wetlands
arcpy.management.Dissolve("SE_Wetlands_coastal", "SE_Wetlands_coastal_Dissolve", None, None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_Wetlands_coastal_Dissolve'>

In [43]:
# create a 30 m buffer
arcpy.analysis.Buffer("SE_Wetlands_coastal_Dissolve", "SE_Wetlands_coastal_Dissolve_30m", "30 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_Wetlands_coastal_Dissolve_30m'>

In [45]:
# convert poly to raster
arcpy.FeatureToRaster_conversion("SE_Wetlands_coastal_Dissolve_30m", "OBJECTID", "WetlandMask", arcpy.env.cellSize)

<Result 'F:\\Blueprint\\2023Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\WetlandMask'>

### Projection order

In [6]:
# convert poly to raster
arcpy.FeatureToRaster_conversion("SE_Wetlands_coastal_Dissolve_30mAlbers", "OBJECTID", "WetlandMask2", arcpy.env.cellSize)

<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\WetlandMask2'>

In [7]:
# define spatial reference
sr= arcpy.SpatialReference(5070)

arcpy.env.snapRaster = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"
arcpy.env.cellSize = r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

# project rasters
arcpy.management.ProjectRaster("WetlandMask", "WetlandMaskP", sr, "BILINEAR", "30 30", "#", "#", "#", "NO_VERTICAL")
#arcpy.management.ProjectRaster("Gsd", "GsdP", sr,  "BILINEAR", "30 30", "WGS_1984_(ITRF00)_To_NAD_1983", "#", "#", "NO_VERTICAL")


<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\WetlandMaskP'>

# Extract Stable marsh from coastal marsh mask

In [34]:
# intersect wetland mask with full stable marsh files

ATL_StableMarsh_clip = arcpy.sa.Times("ATL_StableMarsh_all_SE", "WetlandMask2"); ATL_StableMarsh_clip.save("ATL_StableMarsh_clip")

GOM_StableMarsh_clip = arcpy.sa.Times("GOM_StableMarsh_allP", "WetlandMask2"); GOM_StableMarsh_clip.save("GOM_StableMarsh_clip")

In [36]:
arcpy.management.MosaicToNewRaster("ATL_StableMarsh_clip;GOM_StableMarsh_clip", env.workspace, "SE_stable_marsh", sr, "8_BIT_UNSIGNED", 30, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_stable_marsh'>

In [37]:
# reclass to Score = 2
# set up local variables
remap = RemapValue([[1, 2]])
# Execute Reclassify
SE_stable_marsh2 = Reclassify("SE_stable_marsh", "VALUE", remap)
SE_stable_marsh2.save("SE_stable_marsh2")

## Unstable/stable low certainity marsh

In [49]:
# intersect wetland mask with full costal marsh files

SE_CoastalMarsh = arcpy.sa.Times("SE_extent", "WetlandMask2"); SE_CoastalMarsh.save("SE_CoastalMarsh")

remap = RemapValue([[0, 1]])
# Execute Reclassify
SE_CoastalMarsh1 = Reclassify("SE_CoastalMarsh", "VALUE", remap)
SE_CoastalMarsh1.save("SE_CoastalMarsh1")

## Stack up raster

In [50]:
# Execute CellStatistics
SE_marsh = CellStatistics(["SE_stable_marsh2", "SE_CoastalMarsh1", "SE_extent"], "MAXIMUM", "DATA", "SINGLE_BAND")

# Save the output 
SE_marsh.save("SE_marsh")

## Finalize indicator

In [51]:
# clip to SE BP boundary
SE_marsh_clip = arcpy.sa.ExtractByMask("SE_marsh", extent); SE_marsh_clip.save("SE_marsh_clip")

In [ ]:
## UPDATE CODE FORE MULTI CLASS RASTER

In [2]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return '2 = Stable coastal wetlands'
    elif value == 1:
        return '1 = Other coastal wetlands' 
    elif value == 0:
        return '0 = Not identified as coastal wetlands'  
"""

In [3]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("SE_marsh_clip", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'C:\\Data\\Indicators\\CoastalMarsh\\CoastalMarsh2023.gdb\\SE_marsh_clip'>

In [2]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 29
	if Value == 1:
		return 157
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 75
	if Value == 1:
		return 164
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 168
	if Value == 1:
		return 233
	else:
		return 255
"""

In [3]:
# calculate Red field
arcpy.management.CalculateField("SE_marsh_clip", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("SE_marsh_clip", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("SE_marsh_clip", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'SE_marsh_clip'>

In [6]:
# clip to 2023 VIPR Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=extent, snapRaster=extent, cellSize=extent):
    SEMask = arcpy.sa.ExtractByMask("SE_marsh_clip", extent); SEMask.save("SEMask")

In [9]:
# export to tiff
with arcpy.EnvManager(snapRaster=extent, extent=extent, cellSize=extent):
    arcpy.management.CopyRaster("SEMask", SEout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

## FIN